## Status Invest

This script reads all stocks data available in https://statusinvest.com.br/ using scrapers.\
Giving you the possibility to filter out only the relevant stocks as desired.

In [18]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from libs.scrapers.statusinvest import (
    StockListScraper,
)

### Reading List of Stocks (plus general stock data)

In [19]:
stocks_general_data_df = StockListScraper().dataframe_format

# show most liquid stocks (it also shows only the first X columns)
stocks_general_data_df.sort_values("LIQUIDEZ MEDIA DIARIA", ascending=False, inplace=True)
print(stocks_general_data_df[stocks_general_data_df.columns[0:10]][0:5].to_string(index=False))
print(f"--- Total stock tickers in the list: {stocks_general_data_df.shape[0]} ---")

TICKER  PRECO    DY   P/L  P/VP  P/ATIVOS  MARGEM BRUTA  MARGEM EBIT  MARG. LIQUIDA  P/EBIT
 VALE3  69.50  8.75  6.64  1.66      0.71         40.12        31.07          23.03    4.92
 PETR4  39.96 18.31  3.81  1.35      0.51         51.08        38.69          25.53    2.51
 ITUB4  32.81  3.83 10.10  1.75      0.13         30.03         9.92           8.30    8.46
 BBDC4  15.52  7.17 11.93  0.99      0.09         30.94         3.95           6.18   18.68
 BBAS3  56.97  8.03  4.90  0.99      0.07         32.66        15.94          12.40    3.81
--- Total stock tickers in the list: 620 ---


### Filter out stocks

In [20]:
# by trading liquidity
stocks_general_data_df = stocks_general_data_df[stocks_general_data_df["LIQUIDEZ MEDIA DIARIA"] >= 1_000_000]

# filter out only the most liquid stock from each company
stocks_general_data_df["PREFIX"] = stocks_general_data_df["TICKER"].str.upper().str[:4]
stocks_general_data_df.sort_values("LIQUIDEZ MEDIA DIARIA", ascending=False, inplace=True)
stocks_general_data_df.drop_duplicates("PREFIX", keep="first", inplace=True)
stocks_general_data_df.drop("PREFIX", axis=1, inplace=True)

# show stock tickers count after filter applied
print(f"--- Filtered stock tickers in the list: {stocks_general_data_df.shape[0]} ---")

--- Filtered stock tickers in the list: 219 ---


### Output result into a file

In [3]:
stocks_general_data_df.to_csv("../outputs/statusinvest.csv", sep=";", index=False)